In [1]:
# ensure you have necessary packages 
import transformers 
import datasets 
import torch
import pandas as pd

In [2]:
pip install transformers

     ---------------------------------------- 4.9/4.9 MB 2.6 MB/s eta 0:00:00
     -------------------------------------- 267.7/267.7 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 120.7/120.7 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 140.4/140.4 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install datasets

     -------------------------------------- 431.2/431.2 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 10.4/10.4 MB 7.5 MB/s eta 0:00:00
     -------------------------------------- 319.7/319.7 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 133.1/133.1 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 19.5/19.5 MB 7.5 MB/s eta 0:00:00
     -------------------------------------- 140.8/140.8 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 95.8/95.8 kB 5.3 MB/s eta 0:00:00
     -------------------------------------- 55.9/55.9 kB 972.3 kB/s eta 0:00:00
     -------------------------------------- 500.6/500.6 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install beir

     ---------------------------------------- 50.3/50.3 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 9.9/9.9 MB 6.1 MB/s eta 0:00:00
     -------------------------------------- 219.2/219.2 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 7.4/7.4 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 38.6/38.6 MB 2.2 MB/s eta 0:00:00
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
     ---------------------------------------- 1.1/1.1 MB 4.8 MB/s eta 0:00:00
  Using cach

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [13 lines of output]
  Fetching trec_eval from https://github.com/usnistgov/trec_eval/archive/v9.0.8.tar.gz.
  C:\Users\dabho\anaconda3\envs\uiuc_chatbot\lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
    warnings.warn(
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pytrec_eval
  copying py\__init__.py -> build\lib.win-amd64-cpython-310\pytrec_eval
  running build_ext
  building 'pytrec_eval_ext' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates

In [5]:
pip install torch

     -------------------------------------- 162.2/162.2 MB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Try OPT-125m (smallest)

In [7]:
from transformers import GPT2Tokenizer, OPTForCausalLM

model = OPTForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-125m")

prompt = "Hey, are you consciours? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
generated_sentence = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(generated_sentence)

Hey, are you consciours? Can you talk to me?
I'm not sure, but I'm sure I can talk to you


# Try Contriever 

First clone the contriever repo into the current directory. 

When pushing, **do not** push this directory.

```bash
git clone git@github.com:facebookresearch/contriever.git
```

In [6]:
!git clone git@github.com:facebookresearch/contriever.git

Cloning into 'contriever'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [9]:
import sys
# print(sys.path)
sys.path.append("./contriever")

from src.contriever import Contriever
from transformers import AutoTokenizer

model = Contriever.from_pretrained("facebook/contriever") 
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever") #Load the associated tokenizer:


sentences = [
    "Where was Marie Curie born?",
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings = model(**inputs)


score01 = embeddings[0] @ embeddings[1] #1.0473
score02 = embeddings[0] @ embeddings[2] #1.0095

print(score01)
print(score02)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

NameError: name 'Contriever' is not defined

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('facebook/contriever-msmarco')
model = AutoModel.from_pretrained('facebook/contriever-msmarco')

sentences = [
    "Where was Marie Curie born?",
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]

# Apply tokenizer
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
outputs = model(**inputs)

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings
embeddings = mean_pooling(outputs[0], inputs['attention_mask'])


OSError: Windows requires Developer Mode to be activated, or to run Python as an administrator, in order to create symlinks.
In order to activate Developer Mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development

In [ ]:
embeddings